In [ ]:
using Revise
using Pkg; Pkg.activate(".")

In [ ]:
using Unitful
using PotentialLearning
using Random: randperm
using JLD2
using InteratomicPotentials
using AtomsBase, AtomsCalculators
using Statistics
using CairoMakie, ColorSchemes

In [ ]:
ensemble_members = load("ace_cmte1.jld2", "members")

In [ ]:
includet("../files/committee_potentials.jl")
includet("../files/committee_qois.jl")

In [ ]:
my_cmte = CommitteePotential(ensemble_members; energy_units=u"eV", length_units=u"Å")
cmte_energy = CmteEnergy(Statistics.var, strip_units=true)
# !!!!! important, I changed this to be variance, not std

In [ ]:
datasets = load("datasets_with_descriptors.jld2")
pristine_base_calib_ds = datasets["pristine_base_calib_ds"]
pristine_base_test_ds = datasets["pristine_base_test_ds"]
frenkel_base_calib_ds = datasets["frenkel_base_calib_ds"]
frenkel_base_test_ds = datasets["frenkel_base_test_ds"]

In [ ]:
includet("../files/conformal_prediction_utils.jl")

In [ ]:
# from subsampling_dpp.jl in PL.jl examples
function concat_dataset(confs::Vector{DataSet})
    N = length(confs)
    confs_vec = [[confs[i][j] for j = 1:length(confs[i])] for i = 1:N]
    confs_all = reduce(vcat, confs_vec)
    return DataSet(confs_all)
end

In [ ]:
orig_combined_calib_ds = concat_dataset([pristine_base_calib_ds; frenkel_base_calib_ds])
orig_combined_test_ds = concat_dataset([pristine_base_test_ds; frenkel_base_test_ds])

In [ ]:
rand_idxs = randperm(1500)
combined_calib_ds = orig_combined_calib_ds[rand_idxs]
combined_test_ds = orig_combined_test_ds[rand_idxs]

In [ ]:
cmte_cov_energy = CmteEnergyCov(true)

In [ ]:
# Already random, so just take every two
ediff_combined_calib_ref = Float64[]
ediff_combined_calib_pred = Float64[]
sys1_combined_calib_uqs = Float64[]
sys2_combined_calib_uqs = Float64[]
ediff_combined_calib_cov_uq = Float64[]

sys1_combined_calib_epreds = Float64[]
sys2_combined_calib_epreds = Float64[]
for i in 1:2:length(combined_calib_ds)
    sys1 = combined_calib_ds[i]
    sys2 = combined_calib_ds[i+1]
    e1_ref = ustrip(get_values(get_energy(sys1)))
    e2_ref = ustrip(get_values(get_energy(sys2)))

    push!(ediff_combined_calib_ref, e2_ref - e1_ref)

    e1_pred = ustrip(PotentialLearning.potential_energy(sys1, my_cmte))
    push!(sys1_combined_calib_epreds,e1_pred)
    e2_pred = ustrip(PotentialLearning.potential_energy(sys2, my_cmte))
    sys2_epreds = push!(sys2_combined_calib_epreds,e2_pred)

    push!(ediff_combined_calib_pred, e2_pred - e1_pred)

    sys1_uq = ustrip(compute(cmte_energy,sys1,my_cmte))
    push!(sys1_combined_calib_uqs,sys1_uq)
    sys2_uq = ustrip(compute(cmte_energy,sys2,my_cmte))
    push!(sys2_combined_calib_uqs,sys2_uq)

    cov_uq = ustrip(compute(cmte_cov_energy,sys1,sys2,my_cmte; flip_second_sign=true))
    push!(ediff_combined_calib_cov_uq, cov_uq)

end

In [ ]:
# Var(X1) + Var(-X2) + 2*Cov(X1,-X2), note Var(-X2) = Var(X2)
calib_ediff_uq = sqrt.(sys1_combined_calib_uqs .+ sys2_combined_calib_uqs .+ 2*ediff_combined_calib_cov_uq)

In [ ]:
large_pristine_ds = load("large_pristine_ds.jld2", "large_pristine_ds")
large_8x_frenkel_ds = load("large_8x_frenkel_ds.jld2", "large_8x_frenkel_ds")
large_dilute_frenkel_ds = load("large_dilute_frenkel_ds.jld2", "large_dilute_frenkel_ds")

In [ ]:
function custom_histogram2(data1, data2;
    width=600,
    bins=10,
    colormap=:viridis,
    color_value=0.0,
    title="Histogram",
    xlabel="Score Values",
    ylabel="Frequency",
    fill_alpha=0.5,
    edge_linewidth=1.0,
    axis_color=:black,
    text_size=18,
    label_size=22,
    grid_visible=false,
    grid_color=(:gray, 0.3),
    grid_linewidth=0.5,
    bar_color=nothing,
    edge_color=nothing,
    normalize=false,
    kde=false,
    kde_linewidth=3.0,
    kde_color=:black)

# Get color from colormap if no specific colors provided
base_color1 = get(ColorSchemes.colorschemes[colormap], color_value+0.3)
bar_color1 = isnothing(bar_color) ? (base_color1, 0.8) : bar_color

base_color2 = get(ColorSchemes.colorschemes[colormap], color_value)
bar_color2 = isnothing(bar_color) ? (base_color2, fill_alpha) : bar_color

#base_color1 = :black
#bar_color1 = (:black, fill_alpha)
#
#base_color2 = :black
#bar_color2 = (:black, fill_alpha)

#edge_color = isnothing(edge_color) ? darker(base_color, 0.2) : edge_color

# Create figure and axis with better formatting
fig = Figure(resolution=(width, width), fontsize=text_size)

# Calculate sensible limits with buffer
#data1 = [datum for datum in data1 if datum < 2]
#data2 = [datum for datum in data2 if datum < 2]
data_min = minimum([data1;data2])
data_max = maximum([data1;data2])
#buffer = (data_max - data_min) * 0.05
#x_min = data_min - buffer
#x_max = data_max + buffer
x_min = data_min
x_max = data_max
#x_min = -0.01
#x_max = 1.0

# Create axis with formatting
ax = Axis(fig[1, 1],
#title=title,
xlabel=xlabel,
ylabel=ylabel,
xlabelsize=label_size,
ylabelsize=label_size,
titlesize=label_size,
xticklabelsize=text_size,
yticklabelsize=text_size,
spinewidth=1.5,
xgridvisible=grid_visible,
ygridvisible=grid_visible,
xgridcolor=grid_color,
ygridcolor=grid_color,
xgridwidth=grid_linewidth,
ygridwidth=grid_linewidth
)

# Set spine and tick colors

ax.bottomspinecolor = axis_color
ax.leftspinecolor = axis_color
ax.rightspinecolor = axis_color
ax.topspinecolor = axis_color

#ax.xticks = 0:0.2:1.0
#ax.yticks = 0:10:50

ax.xticklabelcolor = axis_color
ax.yticklabelcolor = axis_color
ax.xlabelcolor = axis_color
ax.ylabelcolor = axis_color
ax.titlecolor = axis_color

# Add extra padding to avoid cutting off labels
#fig.margin = 20

# Create the histogram
hist = hist!(ax, data1,
bins=bins,
color=bar_color1,
#strokecolor=edge_color,
strokecolor=bar_color1,
strokewidth=edge_linewidth,
normalization=normalize ? :pdf : :none)

hist = hist!(ax, data2,
bins=bins,
color=bar_color2,
#strokecolor=edge_color,
strokecolor=bar_color2,
strokewidth=edge_linewidth,
normalization=normalize ? :pdf : :none)

# Optionally add KDE curve
#if kde
#density = kde!(ax, data,
#color=kde_color,
#linewidth=kde_linewidth,
#label="KDE")
#
## Add legend if KDE is used
#axislegend(ax, position=:rt, framevisible=true,
#framecolor=(:black, 0.2),
#padding=(10, 10, 10, 10),
#labelsize=text_size-2)
#end

# Adjust x limits
ax.limits = (x_min, x_max, nothing, nothing)

return fig
end

In [ ]:
calib_residuals = abs.(ediff_combined_calib_pred .- ediff_combined_calib_ref)


In [ ]:
pristine_idxs = randperm(102)

index_pairs = Tuple{Int64,Int64}[]
for i in 1:102
    pi = pristine_idxs[i]
    large_8x_idxs = randperm(102)
    for j in 1:10
        push!(index_pairs, (pi, large_8x_idxs[j]))
    end
end



In [ ]:
# pristine, 8x
p8only_ediff_combined_test_ref = Float64[]
p8only_ediff_combined_test_pred = Float64[]
p8only_sys1_combined_test_uqs = Float64[]
p8only_sys2_combined_test_uqs = Float64[]
p8only_ediff_combined_test_cov_uq = Float64[]

p8only_sys1_combined_test_epreds = Float64[]
p8only_sys2_combined_test_epreds = Float64[]
for i in eachindex(index_pairs)
    pi = index_pairs[i][1]
    fi = index_pairs[i][2]
    sys1 = large_pristine_ds[pi]
    sys2 = large_8x_frenkel_ds[fi]
    e1_ref = ustrip(get_values(get_energy(sys1)))
    e2_ref = ustrip(get_values(get_energy(sys2)))

    push!(p8only_ediff_combined_test_ref, e2_ref - e1_ref)

    e1_pred = ustrip(PotentialLearning.potential_energy(sys1, my_cmte))
    push!(p8only_sys1_combined_test_epreds,e1_pred)
    e2_pred = ustrip(PotentialLearning.potential_energy(sys2, my_cmte))
    sys2_epreds = push!(p8only_sys2_combined_test_epreds,e2_pred)

    push!(p8only_ediff_combined_test_pred, e2_pred - e1_pred)

    sys1_uq = ustrip(compute(cmte_energy,sys1,my_cmte))
    push!(p8only_sys1_combined_test_uqs,sys1_uq)
    sys2_uq = ustrip(compute(cmte_energy,sys2,my_cmte))
    push!(p8only_sys2_combined_test_uqs,sys2_uq)

    cov_uq = ustrip(compute(cmte_cov_energy,sys1,sys2,my_cmte; flip_second_sign=true))
    push!(p8only_ediff_combined_test_cov_uq, cov_uq)
end

In [ ]:
p8only_test_ediff_uq = sqrt.(p8only_sys1_combined_test_uqs .+ p8only_sys2_combined_test_uqs .+ 2*p8only_ediff_combined_test_cov_uq)
ediff_combined_calib_scores = abs.(ediff_combined_calib_pred .- ediff_combined_calib_ref) ./ calib_ediff_uq
p8only_test_abs_residuals_combined = abs.(p8only_ediff_combined_test_pred .- p8only_ediff_combined_test_ref)
alpha_complements = collect(range(0.01,0.99,step=0.01))
alpha_refs = 1 .- alpha_complements

#p8only_alpha_pred, debug1, debug2 = generate_predicted_alphas(ediff_combined_calib_scores,p8only_test_ediff_uq, p8only_test_abs_residuals_combined)
p8only_alpha_pred = generate_predicted_alphas(ediff_combined_calib_scores,p8only_test_ediff_uq, p8only_test_abs_residuals_combined)

In [ ]:
qhat= calibrate(ediff_combined_calib_pred,ediff_combined_calib_ref,calib_ediff_uq, 0.2)

qhat_uq = qhat*p8only_test_ediff_uq

In [ ]:
p8only_scores = p8only_test_abs_residuals_combined ./p8only_test_ediff_uq
#p8hist_fig = custom_histogram2(ediff_combined_calib_scores, p8only_scores, bins=50, colormap=:lipari, text_size=24, label_size=28)
#save("p8hist.svg", p8hist_fig)

#p8hist_fig = custom_histogram2([datum for datum in calib_ediff_uq if datum < 0.5],
#                                [datum for datum in p8only_test_ediff_uq if datum < 0.5], bins=50, colormap=:roma, text_size=24, label_size=28)
#
p8hist_fig = custom_histogram2([datum for datum in p8only_test_ediff_uq[1:750] if datum < 10.0],
                                [datum for datum in calib_ediff_uq if datum < 10.0],
                                 bins=50, colormap=:roma, text_size=28, label_size=32, xlabel="Uncertainty Estimate (eV)")

In [ ]:
score_fig = custom_histogram2(ediff_combined_calib_scores,
                                p8only_scores,
                                 bins=50, colormap=:roma, text_size=28, label_size=32, xlabel="Uncertainty Estimate (eV)")

In [ ]:
pristine_idxs = randperm(102)

index_pairs2 = Tuple{Int64,Int64}[]
for i in 1:102
    pi = pristine_idxs[i]
    large_dilute_idxs = randperm(102)
    for j in 1:10
        push!(index_pairs2, (pi, large_dilute_idxs[j]))
    end
end

In [ ]:
# pristine, dilute
pdonly_ediff_combined_test_ref = Float64[]
pdonly_ediff_combined_test_pred = Float64[]
pdonly_sys1_combined_test_uqs = Float64[]
pdonly_sys2_combined_test_uqs = Float64[]
pdonly_ediff_combined_test_cov_uq = Float64[]

pdonly_sys1_combined_test_epreds = Float64[]
pdonly_sys2_combined_test_epreds = Float64[]
for i in eachindex(index_pairs2)
    pi = index_pairs2[i][1]
    fi = index_pairs2[i][2]
    sys1 = large_pristine_ds[pi]
    sys2 = large_dilute_frenkel_ds[fi]
    e1_ref = ustrip(get_values(get_energy(sys1)))
    e2_ref = ustrip(get_values(get_energy(sys2)))

    push!(pdonly_ediff_combined_test_ref, e2_ref - e1_ref)

    e1_pred = ustrip(PotentialLearning.potential_energy(sys1, my_cmte))
    push!(pdonly_sys1_combined_test_epreds,e1_pred)
    e2_pred = ustrip(PotentialLearning.potential_energy(sys2, my_cmte))
    sys2_epreds = push!(pdonly_sys2_combined_test_epreds,e2_pred)

    push!(pdonly_ediff_combined_test_pred, e2_pred - e1_pred)

    sys1_uq = ustrip(compute(cmte_energy,sys1,my_cmte))
    push!(pdonly_sys1_combined_test_uqs,sys1_uq)
    sys2_uq = ustrip(compute(cmte_energy,sys2,my_cmte))
    push!(pdonly_sys2_combined_test_uqs,sys2_uq)

    cov_uq = ustrip(compute(cmte_cov_energy,sys1,sys2,my_cmte; flip_second_sign=true))
    push!(pdonly_ediff_combined_test_cov_uq, cov_uq)
end

In [ ]:
pdonly_test_ediff_uq = sqrt.(pdonly_sys1_combined_test_uqs .+ pdonly_sys2_combined_test_uqs .+ 2*pdonly_ediff_combined_test_cov_uq)
ediff_combined_calib_scores = abs.(ediff_combined_calib_pred .- ediff_combined_calib_ref) ./ calib_ediff_uq
pdonly_test_abs_residuals_combined = abs.(pdonly_ediff_combined_test_pred .- pdonly_ediff_combined_test_ref)
alpha_complements = collect(range(0.01,0.99,step=0.01))
alpha_refs = 1 .- alpha_complements

#pdonly_alpha_pred, debug1, debug2 = generate_predicted_alphas(ediff_combined_calib_scores,pdonly_test_ediff_uq, pdonly_test_abs_residuals_combined)
pdonly_alpha_pred = generate_predicted_alphas(ediff_combined_calib_scores,pdonly_test_ediff_uq, pdonly_test_abs_residuals_combined)

In [ ]:
pdonly_scores = pdonly_test_abs_residuals_combined ./pdonly_test_ediff_uq
#p8hist_fig = custom_histogram2(ediff_combined_calib_scores, pdonly_scores, bins=50, colormap=:lipari, text_size=24, label_size=28)
#save("p8hist.svg", p8hist_fig)

#p8hist_fig = custom_histogram2([datum for datum in calib_ediff_uq if datum < 0.5],
#                                [datum for datum in pdonly_test_ediff_uq if datum < 0.5], bins=50, colormap=:roma, text_size=24, label_size=28)
#
pdhist_fig = custom_histogram2([datum for datum in pdonly_test_ediff_uq[1:750] if datum < 10.0],
                                [datum for datum in calib_ediff_uq if datum < 10.0],
                                 bins=50, colormap=:roma, text_size=28, label_size=32, xlabel="Uncertainty Estimate (eV)")
#save("pdhist_uq.svg", pdhist_fig)